# Script test

In [1]:
! ./convert_source.py -h

usage: convert_source.py [-h] -s subject_ID -o Output_BIDS_Directory -d
                         data_directory -c config.yml -f file_type
                         [-ses session] [-k] [-v]

Performs conversion of source DICOM, PAR REC, and Nifti data to BIDS directory
layout.

optional arguments:
  -h, --help            show this help message and exit

Required arguments:
  -s subject_ID, -sub subject_ID, --sub subject_ID
                        Unique subject identifier given to each participant.
                        This indentifier CAN contain letters and numbers. This
                        identifier CANNOT contain: underscores, hyphens,
                        colons, semi-colons, spaces, or any other special
                        characters.
  -o Output_BIDS_Directory, -out Output_BIDS_Directory, --out Output_BIDS_Directory
                        BIDS output directory. This directory does not need to
                        exist at runtime. The resulting directory will b

In [2]:
sub = 'C10'
ddir = "/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC"
bids = "/Users/brac4g/Desktop/convsauce/BIDS.new"
conf = "/Users/brac4g/Desktop/convsauce/convert_source/config.default.yml"
ses = 1

In [7]:
! ./convert_source.py -s "{sub}" -o "{bids}" -d "{ddir}" -c "{conf}" -f "par" -ses "{ses}"

Chris Rorden's dcm2niiX version v1.0.20190720  (JP2:OpenJPEG) (JP-LS:CharLS) Clang8.1.0 (64-bit MacOS)
Done reading PAR header version 4.2, with 40 slices
Philips Scaling Values RS:RI:SS = 5.85739:0:0.0663237 (see PMC3998685)
Convert 1 DICOM as /Users/brac4g/Desktop/convsauce/BIDS.new/sub-C10/tmp_dir1339/tmp_basename4118 (512x512x40x1)
Compress: "//Users/brac4g/bin/dcm2nii/bin/pigz_mricron" -b 960 -n -f -6 "/Users/brac4g/Desktop/convsauce/BIDS.new/sub-C10/tmp_dir1339/tmp_basename4118.nii"
Conversion required 0.340739 seconds (0.103620 for core code).
Chris Rorden's dcm2niiX version v1.0.20190720  (JP2:OpenJPEG) (JP-LS:CharLS) Clang8.1.0 (64-bit MacOS)
Done reading PAR header version 4.2, with 216 slices
Error: Bizarre b-vectors /Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_6_DIR_B0_A_TE88_SENSE_NO_MB_NO_4DYN_5_1.REC
Philips Scaling Values RS:RI:SS = 2.01465:0:0.028553 (see PMC3998685)
Convert 1 DICOM as /Users/brac4g/Desktop/convsauce/BIDS.new/sub-C10/tmp_dir8270/tmp_ba

Philips Scaling Values RS:RI:SS = 4.10281:0:0.00606359 (see PMC3998685)
Convert 1 DICOM as /Users/brac4g/Desktop/convsauce/BIDS.new/sub-C10/tmp_dir6674/tmp_basename8281 (512x512x45x1)
Compress: "//Users/brac4g/bin/dcm2nii/bin/pigz_mricron" -b 960 -n -f -6 "/Users/brac4g/Desktop/convsauce/BIDS.new/sub-C10/tmp_dir6674/tmp_basename8281.nii"
Conversion required 0.250732 seconds (0.124745 for core code).
Chris Rorden's dcm2niiX version v1.0.20190720  (JP2:OpenJPEG) (JP-LS:CharLS) Clang8.1.0 (64-bit MacOS)
Done reading PAR header version 4.2, with 45 slices
Philips Scaling Values RS:RI:SS = 9.2105:0:0.0181626 (see PMC3998685)
Convert 1 DICOM as /Users/brac4g/Desktop/convsauce/BIDS.new/sub-C10/tmp_dir2821/tmp_basename4596 (64x64x45x1)
Compress: "//Users/brac4g/bin/dcm2nii/bin/pigz_mricron" -n -f -6 "/Users/brac4g/Desktop/convsauce/BIDS.new/sub-C10/tmp_dir2821/tmp_basename4596.nii"
Conversion required 0.139813 seconds (0.003350 for core code).
Chris Rorden's dcm2niiX version v1.0.20190720  (JP

In [6]:
! echo "{sub}"

C10


In [1]:
# Import packages and modules
import pydicom
import json
import re
import os
import sys
import shutil
import glob
import random
import subprocess
import pathlib
import yaml
import nibabel as nib
import gzip
import pandas as pd
import numpy as np
import platform
import multiprocessing

In [2]:
# Import third party packages and modules
import convert_source_dcm as cdm
import convert_source_par as csp
import convert_source_nii as csn
import utils

In [3]:
conf = "/Users/brac4g/Desktop/convsauce/convert_source/config.default.yml"
# conf = "C:/Users/smart/Desktop/GitProjects/convsauce/ConvertSource/config.default.yml"

In [4]:
def read_config(config_file, verbose = False):
    '''
    Reads configuration file and creates a dictionary of search terms for 
    certain modalities provided that BIDS modalities are used as keys. If
    exclusions are provided (via the key 'exclude') then an exclusion list is 
    created. Otherwise, 'exclusion_list' is returned as an empty list. If 
    additional settings are specified, they should be done so via the key
    'metadata' to enable writing of additional metadata. Otherwise, an 
    empty dictionary is returned.
    
    Arguments:
        config_file (string): file path to yaml configuration file.
        verbose (boolean): Prints additional information to screen.
    
    Returns: 
        data_map (dict): Nested dictionary of search terms for BIDS modalities
        exclusion_list (list): List of exclusion terms
        meta_dict (dict): Nested dictionary of metadata terms to write to JSON file(s)
    '''
    
    with open(config_file) as file:
        data_map = yaml.safe_load(file)
        if verbose:
            print("Initialized parameters from configuration file")
        
    if any("exclude" in data_map for element in data_map):
        if verbose:
            print("exclusion option implemented")
        exclusion_list = data_map["exclude"]
        del data_map["exclude"]
    else:
        if verbose:
            print("exclusion option not implemented")
        exclusion_list = list()
        
    if any("metadata" in data_map for element in data_map):
        if verbose:
            print("implementing additional settings for metadata")
        meta_dict = data_map["metadata"]
        del data_map["metadata"]
    else:
        if verbose:
            print("no metadata settings")
        meta_dict = dict()
        
    return data_map,exclusion_list,meta_dict

In [5]:
search_dict, exclude_list, meta_dict = read_config(conf,verbose=True)

Initialized parameters from configuration file
exclusion option implemented
implementing additional settings for metadata


In [6]:
def create_file_list(data_dir, file_ext="", order="size"):
    '''
    Creates a file list by globbing a directory for a specific file
    extension and sorting by some determined order. A file list is 
    then returned
    
    Arguments:
        data_dir (string): Absolute path to data directory (must be a directory dump of image data)
        file_ext (string): File extension to glob. Built-in options include:
            - 'par' or 'PAR': Searches for PAR headers
            - 'dcm' or 'DICOM': Searches for DICOM directories, then searches for one file from each DICOM directory
            - 'nii', or 'Nifti': Searches for nifti files (including gzipped nifti files)
        order (string): Order to sort the list. Valid options are: 'size' and 'time':
            - 'size': sorts by file size in ascending order (default)
            - 'time': sorts by file modification time in ascending order
            - 'none': no sorting is applied and the list is generated as the system finds the files
    
    Returns: 
        file_list (list): List of filenames, complete with their absolute paths.
    '''
    
    # Check file extension
    if file_ext != "":
        if file_ext.upper() == "PAR" or file_ext.upper() == "REC":
            file_ext = "PAR"
            file_ext = f".{file_ext.upper()}"
        elif file_ext.lower() == "dcm" or file_ext.upper() == "DICOM":
            file_ext = "dcm"
            file_ext = f".{file_ext.lower()}"
        elif file_ext.lower() == "nii" or file_ext.lower() == "nifti":
            file_ext = "nii"
            file_ext = f".{file_ext.lower()}*" # Add wildcard for globbling gzipped files
        else:
            file_ext = f".{file_ext}"
    
    # Check sort order
    if order.lower() == "size":
        order_key = os.path.getsize
    elif order.lower() == "time":
        order_key = os.path.getmtime
    elif order.lower() == "none":
        order_key=None
    else:
        order_key = os.path.getsize
        print("Unrecognized keyword option. Using default.")
    
    # Create file list
    if file_ext == ".dcm":
        file_list = sorted(cdm.get_dcm_files(data_dir), key=order_key, reverse=False)
    elif file_ext != ".dcm":
        file_names = os.path.join(data_dir, f"*{file_ext}")
        file_list = sorted(glob.glob(file_names, recursive=True), key=order_key, reverse=False)
    
    return file_list

In [7]:
def file_exclude(file_list, data_dir, exclusion_list = [], verbose = False):
    '''
    Excludes files from the conversion process by removing filenames
    that contain words that match those found in the 'exclusion_list'
    from the 'read_config' function - should any files need/want to be 
    excluded.
    
    If 'exclusion_list' is empty, then the original 'file_list' is returned.
    
    Arguments:
        file_list (list): List of filenames
        data_dir (string): Absolute path to parent directory that contains the image data
        exclusion_list (list): List of words to be matched. Filenames that contain these words will be excluded.
        verbose (bool): Boolean - True or False.
    
    Returns: 
        currated_list (list): Currated list of filenames, with unwanted filenames removed.
    '''
            
    # Check file extension in file list
    if 'dcm' in file_list[0]:
        file_ext = "dcm"
        file_ext = f".{file_ext.lower()}"
    elif 'PAR' in file_list[0]:
        file_ext = "PAR"
        file_ext = f".{file_ext.upper()}"
    elif 'nii' in file_list[0]:
        file_ext = "nii"
        file_ext = f".{file_ext.lower()}*" # Add wildcard for globbling gzipped files
    else:
        file_ext = ""
        file_ext = f".{file_ext.lower()}"
    
    # create set of lists
    file_set = set(file_list)
    
    # create empty sets
    currated_set = set()
    exclusion_set = set()
    
    if len(exclusion_list) == 0:
        currated_set = file_set
    else:
        for file in exclusion_list:
            if file_ext == '.dcm':
                dir_ = os.path.join(data_dir, f"*{file}*",f"*{file_ext}")
            else:
                dir_ = os.path.join(data_dir, f"*{file}*{file_ext}")
            f_names = glob.glob(dir_, recursive=True)        
            f_names_set = set(f_names)
            if verbose:
                if len(f_names_set) != 0:
                    print(f"Excluded files: {f_names_set} \n")
            exclusion_set.update(f_names_set)
            
        currated_set = file_set.difference(exclusion_set)

    currated_list = list(currated_set)
    
    return currated_list

In [8]:
dcm_dir = "/Users/brac4g/Desktop/convsauce/IRC287H-8/20171003"
par_dir = "/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC"
nii_dir = "/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI"

# dcm_dir = "C:/Users/smart/Desktop/GitProjects/convsauce/IRC287H-8/20171003"
# par_dir = "C:/Users/smart/Desktop/GitProjects/convsauce/287H_C10/PAR REC"
# nii_dir = "C:/Users/smart/Desktop/GitProjects/convsauce/287H_C10/NIFTI"

# dcm_dir = "C:\\Users\\smart\\Desktop\\GitProjects\\convsauce\\IRC287H-8\\20171003"
# par_dir = "C:\\Users\\smart\\Desktop\\GitProjects\\convsauce\\287H_C10\\PAR REC"
# nii_dir = "C:\\Users\\smart\\Desktop\\GitProjects\\convsauce\\287H_C10\\NIFTI"

In [9]:
dcm_file_list = create_file_list(dcm_dir,'dcm')
par_file_list = create_file_list(par_dir,'par')
nii_file_list = create_file_list(nii_dir,'nii')

In [10]:
dcm_list_currated = file_exclude(dcm_file_list,dcm_dir,exclude_list,verbose=True)
par_list_currated = file_exclude(par_file_list,par_dir,exclude_list,verbose=True)
nii_list_currated = file_exclude(nii_file_list,nii_dir,exclude_list,verbose=True)

Excluded files: {'/Users/brac4g/Desktop/convsauce/IRC287H-8/20171003/1701_WM_SV_PRESS_35_017100311174543840/MR1701000001.dcm'} 

Excluded files: {'/Users/brac4g/Desktop/convsauce/IRC287H-8/20171003/0_DEFAULT_PS_SERIES_2017100310374358016/PR0000000001.dcm', '/Users/brac4g/Desktop/convsauce/IRC287H-8/20171003/0_DEFAULT_PS_SERIES_2017100310463791022/PR0000000001.dcm'} 

Excluded files: {'/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_NEONATAL_SURVEY_1_1.PAR', '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_NEONATAL_SURVEY_16_1.PAR', '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_NEONATAL_SURVEY_2_1.PAR', '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_NEONATAL_SURVEY_13_1.PAR', '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_NEONATAL_SURVEY_17_1.PAR', '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_NEONATAL_SURVEY_14_1.PAR'} 

Excluded files: {'/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C

In [11]:
len(dcm_list_currated)

3

In [12]:
len(par_list_currated)

21

In [13]:
len(nii_list_currated)

12

In [14]:
# utils.get_metadata(meta_dict,'func')

In [15]:
def get_scan_tech(bids_out_dir, sub, file, search_dict, meta_dict=dict(), ses=1, keep_unknown=True, verbose=False):
    '''
    Searches DICOM or PAR file header for scan technique/MR modality used in accordance with the search terms provided
    by the nested dictionary.
    
    Arguments:
        bids_out_dir (string): Output BIDS directory
        sub (int or string): Subject ID
        file (string): Source image filename with absolute filepath
        search_dict (dict): Nested dictionary from the 'read_config' function
        meta_dict (dict): Nested metadata dictionary
        ses (int or string): Session ID
        keep_unknown (bool): Convert modalities/scans which cannot be identified (default: True)
        verbose (bool): Prints the scan_type, modality, and search terms used (e.g. func - bold - rest - ['rest', 'FFE'])
    
    Returns: 
        None
    '''
    
    if not meta_dict:
        meta_dict = dict()

    converted_files = list()
    
    # Check file extension in file
    # Perform Scanning Techniqe Search
    if '.dcm' in file.lower():
        converted_files = cdm.get_dcm_scan_tech(bids_out_dir=bids_out_dir, sub=sub, dcm_file=file, search_dict=search_dict, meta_dict=meta_dict, ses=1, keep_unknown=keep_unknown, verbose=verbose)
    elif '.PAR' in file.upper():
        converted_files = csp.get_par_scan_tech(bids_out_dir=bids_out_dir, sub=sub, par_file=file, search_dict=search_dict, meta_dict=meta_dict, ses=1, keep_unknown=keep_unknown, verbose=verbose)
    else:
        if verbose:
            print("unknown modality")
        if keep_unknown:
            if verbose:
                print("converting unknown_modality")
            scan_type = 'unknown_modality'
            scan = 'unknown'
            [com_param_dict, scan_param_dict] = utils.get_metadata(dictionary=meta_dict,scan_type=scan_type)
            converted_files = csn.data_to_bids_anat(bids_out_dir=bids_out_dir,file=file,sub=sub,scan=scan,meta_dict_com=com_param_dict,meta_dict_anat=scan_param_dict,ses=ses,scan_type=scan_type)
        
    return converted_files

In [16]:
def convert_modality(bids_out_dir, sub, file, search_dict, meta_dict=dict(), ses=1, keep_unknown=True, verbose=False):
    '''
    Converts an image file and extracts information from the filename (such as the modality). 
    
    Note: This function is still undergoing active development.
    Note: Add support for extra dictionaries
    
    Arguments:
        bids_out_dir (string): Output BIDS directory
        sub (int or string): Subject ID
        file (string): Source image filename with absolute filepath
        search_dict (dict): Nested dictionary from the 'read_config' function
        meta_dict (dict): Nested metadata dictionary
        ses (int or string): Session ID
        keep_unknown (bool): Convert modalities/scans which cannot be identified (default: True)
        verbose (bool): Prints the scan_type, modality, and search terms used (e.g. func - bold - rest - ['rest', 'FFE'])
    
    
    Returns: 
        None
    '''
    
    mod_found = False

    converted_files = list()
    
    # Check file type
    if 'nii' in file:
        file_ext = "nii"
        file_ext = f".{file_ext.lower()}"
    elif 'dcm' in file:
        file_ext = "dcm"
        file_ext = f".{file_ext.lower()}"
        if not cdm.is_valid_dcm(file,verbose):
            sys.exit(f"Invalid DICOM file. Please check {file}")
    
    for key,item in search_dict.items():
        for dict_key,dict_item in search_dict[key].items():
            if isinstance(dict_item,list):
                if utils.list_in_substr(dict_item,file):
                    mod_found = True
                    if verbose:
                        print(f"{key} - {dict_key}: {dict_item}")
                    scan_type = key
                    scan = dict_key
                    [com_param_dict, scan_param_dict] = utils.get_metadata(dictionary=meta_dict,scan_type=scan_type)
                    if scan_type.lower() == 'dwi':
                        converted_files = csn.data_to_bids_dwi(bids_out_dir=bids_out_dir,file=file,sub=sub,scan=scan,meta_dict_com=com_param_dict,meta_dict_dwi=scan_param_dict,ses=ses,scan_type=scan_type)
                    elif scan_type.lower() == 'fmap':
                        converted_files = csn.data_to_bids_fmap(bids_out_dir=bids_out_dir,file=file,sub=sub,scan=scan,meta_dict_com=com_param_dict,meta_dict_fmap=scan_param_dict,ses=ses,scan_type=scan_type)
                    else:
                        converted_files = csn.data_to_bids_anat(bids_out_dir=bids_out_dir,file=file,sub=sub,scan=scan,meta_dict_com=com_param_dict,meta_dict_anat=scan_param_dict,ses=ses,scan_type=scan_type)
                    if mod_found:
                        break
            elif isinstance(dict_item,dict):
                tmp_dict = search_dict[key]
                for d_key,d_item in tmp_dict[dict_key].items():
                    if utils.list_in_substr(d_item,file):
                        mod_found = True
                        if verbose:
                            print(f"{key} - {dict_key} - {d_key}: {d_item}")
                        scan_type = key
                        scan = dict_key
                        task = d_key
                        [com_param_dict, scan_param_dict] = utils.get_metadata(dictionary=meta_dict,scan_type=scan_type,task=task)
                        if scan_type.lower() == 'func':
                            converted_files = csn.data_to_bids_func(bids_out_dir=bids_out_dir,file=file,sub=sub,scan=scan,task=task,meta_dict_com=com_param_dict,meta_dict_func=scan_param_dict,ses=ses,scan_type=scan_type)
                        elif scan_type.lower() == 'dwi':
                            converted_files = csn.data_to_bids_dwi(bids_out_dir=bids_out_dir,file=file,sub=sub,scan=scan,meta_dict_com=com_param_dict,meta_dict_dwi=scan_param_dict,ses=ses,scan_type=scan_type)
                        elif scan_type.lower() == 'fmap':
                            converted_files = csn.data_to_bids_fmap(bids_out_dir=bids_out_dir,file=file,sub=sub,scan=scan,meta_dict_com=com_param_dict,meta_dict_fmap=scan_param_dict,ses=ses,scan_type=scan_type)
                        else:
                            converted_files = csn.data_to_bids_anat(bids_out_dir=bids_out_dir,file=file,sub=sub,scan=scan,meta_dict_com=com_param_dict,meta_dict_anat=scan_param_dict,ses=ses,scan_type=scan_type)
                        if mod_found:
                            break
                        
    if not mod_found:
        converted_files = get_scan_tech(bids_out_dir, sub, file, search_dict, meta_dict="", ses=1, keep_unknown=keep_unknown, verbose=verbose)
    
    return converted_files

In [17]:
def batch_convert(bids_out_dir,sub,file_list, search_dict, meta_dict=dict(), ses=1, keep_unknown=True,verbose=False):
    '''
    Batch conversion function for image files.
    
    Note: This function is still undergoing active development.

    Arguments:
        bids_out_dir (string): Output BIDS directory
        sub (int or string): Subject ID
        file (string): Source image filename with absolute filepath
        search_dict (dict): Nested dictionary from the 'read_config' function
        meta_dict (dict): Nested metadata dictionary
        ses (int or string): Session ID
        keep_unknown (bool): Convert modalities/scans which cannot be identified (default: True)
        verbose (bool): Prints the scan_type, modality, and search terms used (e.g. func - bold - rest - ['rest', 'FFE'])

    Returns: 
        None
    '''

    converted_files = list()
    
    for file in file_list:
        try:
            converted_files = convert_modality(bids_out_dir=bids_out_dir, sub=sub, file=file, search_dict=search_dict, meta_dict=meta_dict, ses=1, keep_unknown=True, verbose=False)
        except (SystemExit,FileNotFoundError):
            pass
    
    return converted_files

# Unit Test

In [18]:
# bids_dir = "C:\\Users\\smart\\Desktop\\GitProjects\\convsauce\\BIDS.new"
bids_dir = "/Users/brac4g/Desktop/convsauce/BIDS.new"
# sub = 'P08'
sub = 'C10'
ses = 1

In [20]:
par_list_currated 

['/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_6_DIR_B0_A_TE88_6DYN_20_1.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_CORONAL__3_2.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_T2W_TSE_AXIAL_NEONATE_NSA1_15_1.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_SWIp_11_1.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_SAG_15_5.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_Philips_GRE_Map_SENSE_10_1.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_SAG_T1W_3D_Y_INNER_TI_1100_3_1.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_AXIAL__4_4.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_SAG_4_5.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_DTI_B2000_A68_FAT_SHIFT_P_MB2_SENSE_2_TE88_21_1.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_DTI_6DIR_B800_12_1.PAR',
 '/Users/bra

In [21]:
dcm_list_currated

['/Users/brac4g/Desktop/convsauce/IRC287H-8/20171003/1101_rsfMRI_MB6_SENSE_1_fat_shift_P_017100310465322437/MR1101027463.dcm',
 '/Users/brac4g/Desktop/convsauce/IRC287H-8/20171003/303_CORONAL_2017100310262626000/MR0303000091.dcm',
 '/Users/brac4g/Desktop/convsauce/IRC287H-8/20171003/201_SAG_T1W_3D_Y_INNER_TI_1100_017100310184810020/MR0201000137.dcm']

In [22]:
nii_list_currated

['/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_AXIAL__15_4.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_CORONAL__4_3.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_WIP_T2W_TSE_AXIAL_NEONATE_NSA1_4_1.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_SAG_15_5.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_CORONAL__15_3.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_AXIAL__4_4.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_CORONAL__3_2.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_WIP_SAG_T1W_3D_Y_INNER_TI_1100_3_1.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_WIP_Philips_GRE_Map_SENSE_10_1.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_AXIAL__3_3.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_SAG_4_5.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_WIP_T2W_TSE_A

# Unit Tests

## PAR REC

### DWI

#### Single-band reference (`sbref`, or reverse phase encoded (rPE) B$_{0}$) 
--               
Grade: Passed

In [22]:
par_list_currated[7]

'/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_6_DIR_B0_A_TE88_SENSE_NO_MB_NO_4DYN_5_1.PAR'

In [23]:
# csn.data_to_bids_dwi(bids_out_dir=bids_dir,
#                  sub=sub,
#                  file=par_list_currated[0],
#                  ses=ses)

In [23]:
convert_modality(bids_out_dir=bids_dir,
                 sub=sub,
                 file=par_list_currated[7],
                 search_dict=search_dict,
                 meta_dict=meta_dict,
                 ses=ses,
                 keep_unknown=True,
                 verbose=True)

dwi - dwi: ['diffusion', 'DTI', 'DWI', '6_DIR']


#### DWI Data
--               
Grade: Passed

In [25]:
par_list_currated[3]

'/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_DTI_B2000_A68_FAT_SHIFT_P_MB2_SENSE_2_TE88_21_1.PAR'

In [26]:
convert_modality(bids_out_dir=bids_dir,
                 sub=sub,
                 file=par_list_currated[3],
                 search_dict=search_dict,
                 meta_dict=meta_dict,
                 ses=ses,
                 keep_unknown=True,
                 verbose=True)

dwi - dwi: ['diffusion', 'DTI', 'DWI', '6_DIR']


('/Users/brac4g/Desktop/convsauce/BIDS.new/sub-P08/ses-001/dwi/sub-P08_ses-001_acq-b2000TE88_dir-PA_run-01_dwi.nii.gz',
 '/Users/brac4g/Desktop/convsauce/BIDS.new/sub-P08/ses-001/dwi/sub-P08_ses-001_acq-b2000TE88_dir-PA_run-01_dwi.json',
 '/Users/brac4g/Desktop/convsauce/BIDS.new/sub-P08/ses-001/dwi/sub-P08_ses-001_acq-b2000TE88_dir-PA_run-01_dwi.bval',
 '/Users/brac4g/Desktop/convsauce/BIDS.new/sub-P08/ses-001/dwi/sub-P08_ses-001_acq-b2000TE88_dir-PA_run-01_dwi.bvec')

### Functional MR

#### Single-band reference (`sbref`) 
--               
Grade: Passed

In [27]:
par_list_currated[12]

'/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_rsfMRI_NR1_MB3_SENSE_1_fat_shift_P_8_1.PAR'

In [28]:
convert_modality(bids_out_dir=bids_dir,
                 sub=sub,
                 file=par_list_currated[12],
                 search_dict=search_dict,
                 meta_dict=meta_dict,
                 ses=ses,
                 keep_unknown=True,
                 verbose=True)

func - bold - rest: ['rsfMR', 'rest', 'FFE', 'FEEPI']


('/Users/brac4g/Desktop/convsauce/BIDS.new/sub-P08/ses-001/func/sub-P08_ses-001_task-rest_dir-PA_run-01_sbref.nii.gz',
 '/Users/brac4g/Desktop/convsauce/BIDS.new/sub-P08/ses-001/func/sub-P08_ses-001_task-rest_dir-PA_run-01_sbref.json')

#### EPI (`bold`) 
--               
Grade: Passed

In [29]:
par_list_currated[-1]

'/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_rsfMRI_400M_MB3_SENSE_1_fat_shift_P_9_1.PAR'

In [30]:
convert_modality(bids_out_dir=bids_dir,
                 sub=sub,
                 file=par_list_currated[-1],
                 search_dict=search_dict,
                 meta_dict=meta_dict,
                 ses=ses,
                 keep_unknown=True,
                 verbose=True)

func - bold - rest: ['rsfMR', 'rest', 'FFE', 'FEEPI']


('/Users/brac4g/Desktop/convsauce/BIDS.new/sub-P08/ses-001/func/sub-P08_ses-001_task-rest_dir-PA_run-01_bold.nii.gz',
 '/Users/brac4g/Desktop/convsauce/BIDS.new/sub-P08/ses-001/func/sub-P08_ses-001_task-rest_dir-PA_run-01_bold.json')

### Fieldmaps

#### Fieldmap (`fmap`) 
--               
Grade: Passed

In [31]:
par_list_currated[14]

'/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_Philips_GRE_Map_SENSE_10_1.PAR'

In [32]:
convert_modality(bids_out_dir=bids_dir,
                 sub=sub,
                 file=par_list_currated[14],
                 search_dict=search_dict,
                 meta_dict=meta_dict,
                 ses=ses,
                 keep_unknown=True,
                 verbose=True)

fmap - fmap: ['map']


('/Users/brac4g/Desktop/convsauce/BIDS.new/sub-P08/ses-001/fmap/sub-P08_ses-001_run-01_fmap_fieldmap.nii.gz',
 '/Users/brac4g/Desktop/convsauce/BIDS.new/sub-P08/ses-001/fmap/sub-P08_ses-001_run-01_fmap_magnitude.nii.gz',
 '/Users/brac4g/Desktop/convsauce/BIDS.new/sub-P08/ses-001/fmap/sub-P08_ses-001_run-01_fmap_fieldmap.json',
 '/Users/brac4g/Desktop/convsauce/BIDS.new/sub-P08/ses-001/fmap/sub-P08_ses-001_run-01_fmap_magnitude.json')

## Anatomical Scans

#### SWI (`swi`) 
--               
Grade: Pending

In [22]:
par_list_currated[8]

'/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_SWIp_11_1.PAR'

In [23]:
convert_modality(bids_out_dir=bids_dir,
                 sub=sub,
                 file=par_list_currated[8],
                 search_dict=search_dict,
                 meta_dict=meta_dict,
                 ses=ses,
                 keep_unknown=True,
                 verbose=True)

swi - swi: ['swi']
Error: unable to convert /Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_SWIp_11_1.PAR


In [35]:
utils.convert_image_data(par_list_currated[-6],'test','/Users/brac4g/Desktop/convsauce/BIDS.new/sub-P08/ses-001/swi')

In [24]:
bvals = list()
bvals = 0

In [23]:
# convert_modality(bids_out_dir=bids_dir,
#                  sub=sub,
#                  file=par_list_currated[0],
#                  search_dict=search_dict,
#                  meta_dict=meta_dict,
#                  ses=ses,
#                  keep_unknown=True,
#                  verbose=True)

In [24]:
convert_modality(bids_out_dir=bids_dir,
                 sub=sub,
                 file=par_list_currated[2],
                 search_dict=search_dict,
                 meta_dict=meta_dict,
                 ses=ses,
                 keep_unknown=True,
                 verbose=True)

anat - T2w: ['T2', 'T2w', 'TSE']


In [25]:
convert_modality(bids_out_dir=bids_dir,
                 sub=sub,
                 file=dcm_list_currated[0],
                 search_dict=search_dict,
                 meta_dict=meta_dict,
                 ses=ses,
                 keep_unknown=True,
                 verbose=True)

anat - T1w: ['T1', 'T1w', 'TFE']


('C:\\Users\\smart\\Desktop\\GitProjects\\convsauce\\BIDS.new\\sub-P08\\ses-001\\anat\\sub-P08_ses-P08_run-02_T1w.nii.gz',
 'C:\\Users\\smart\\Desktop\\GitProjects\\convsauce\\BIDS.new\\sub-P08\\ses-001\\anat\\sub-P08_ses-P08_run-02_T1w.json')

In [26]:
convert_modality(bids_out_dir=bids_dir,
                 sub=sub,
                 file=nii_list_currated[0],
                 search_dict=search_dict,
                 meta_dict=meta_dict,
                 ses=ses,
                 keep_unknown=True,
                 verbose=True)

unknown modality
converting unknown_modality


('C:\\Users\\smart\\Desktop\\GitProjects\\convsauce\\BIDS.new\\sub-P08\\ses-001\\unknown_modality\\sub-P08_ses-P08_run-01_unknown.nii.gz',
 'C:\\Users\\smart\\Desktop\\GitProjects\\convsauce\\BIDS.new\\sub-P08\\ses-001\\unknown_modality\\sub-P08_ses-P08_run-01_unknown.json')

In [20]:
utils.convert_anat(par_list_currated[0],bids_dir,'test')

('C:\\Users\\smart\\Desktop\\GitProjects\\convsauce\\BIDS.new\\test.nii.gz',
 'C:\\Users\\smart\\Desktop\\GitProjects\\convsauce\\BIDS.new\\test.json')

In [47]:
utils.convert_image_data(par_list_currated[0],'test',bids_dir)

In [46]:
par_list_currated[0]

'C:\\Users\\smart\\Desktop\\GitProjects\\convsauce\\287H_C10\\PAR REC\\287H_C10_SAG_15_5.PAR'

# Batch Convert Test (PAR REC)

In [26]:
batch_convert(bids_out_dir=bids_dir,
              sub='C10',
              file_list=par_list_currated,
              search_dict=search_dict,
              meta_dict=meta_dict,
              ses=ses,
              keep_unknown=True,
              verbose=True)

Error: unable to convert /Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_SWIp_11_1.PAR


# Batch Convert Test (DICOM, Philips)

In [19]:
batch_convert(bids_out_dir=bids_dir,
              sub='P08',
              file_list=dcm_list_currated,
              search_dict=search_dict,
              meta_dict=meta_dict,
              ses=ses,
              keep_unknown=True,
              verbose=True)

Error: unable to convert /Users/brac4g/Desktop/convsauce/IRC287H-8/20171003/1101_rsfMRI_MB6_SENSE_1_fat_shift_P_017100310465322437/MR1101027463.dcm


# Batch Convert Test (NifTi-2)

In [28]:
batch_convert(bids_out_dir=bids_dir,
              sub='T10',
              file_list=nii_list_currated,
              search_dict=search_dict,
              meta_dict=meta_dict,
              ses=ses,
              keep_unknown=True,
              verbose=True)

('/Users/brac4g/Desktop/convsauce/BIDS.new/sub-T10/ses-001/anat/sub-T10_ses-T10_run-02_T2w.nii.gz',
 '/Users/brac4g/Desktop/convsauce/BIDS.new/sub-T10/ses-001/anat/sub-T10_ses-T10_run-02_T2w.json')

In [41]:
def test(num):
    '''
    test function
    '''
    
    test_list = []
    
    if num == 1:
        j = ["1"]
        test_list = j
    elif num == 2:
        j = ["2","1"]
        test_list = j
        
    return test_list

In [43]:
test(2)

['2', '1']

In [44]:
jt = list()

In [45]:
jt = test(2)
jt

['2', '1']

# Function edits

In [25]:
def data_to_bids_dwi(bids_out_dir, file, sub, scan='dwi', meta_dict_com=dict(), meta_dict_dwi=dict(), ses=1, scan_type='dwi'):
    '''
    Renames converted NifTi-2 files to conform with the BIDS naming convension (in the case of diffuion image files).
    This function accepts any image file (DICOM, PAR REC, and NifTi-2). If the image file is a raw data file (e.g. DICOM, PAR REC)
    it is converted to NifTi first, then renamed. The output BIDS directory need not exist at runtime. If the original
    data format is NifTi, bval and bvec files will be copied over should they exist, otherwise, they will not be
    generated.
    
    Arguments:
        bids_out_dir (string): Path to output BIDS directory. 
        file (string): Filepath to image file.
        sub (int or string): Subject ID
        scan (string): Modality (e.g. dwi, dki, etc)
        meta_dict_com (dict): Metadata dictionary for common image metadata
        meta_dict_dwi (dict): Metadata dictionary for common diffusion image specific metadata
        ses (int or string): Session ID
        scan_type (string): BIDS sub-directory scan type. Valid options include, but are not limited to: anat, func, fmap, dwi (default), etc.
        
    Returns:
        out_nii (string): Absolute filepath to gzipped output diffusion weighted NifTi-2 file
        out_json (string): Absolute filepath to corresponding JSON file
        out_bval (string): Absolute filepath to corresponding b-values file
        out_bvec (string): Absolute filepath to corresponding b-vectors file
    '''

    # Create Output Directory Variables
    # Zeropad subject ID if possible
    try:
        ses = '{:03}'.format(int(ses))
    except ValueError:
        pass
    # Zeropad session ID if possible
    try:
        ses = '{:03}'.format(int(ses))
    except ValueError:
        pass
    
    out_dir = os.path.join(bids_out_dir, f"sub-{sub}", f"ses-{ses}", f"{scan_type}")

    # Make output directory
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    # Get absolute filepaths
    bids_out_dir = os.path.abspath(bids_out_dir)
    out_dir = os.path.abspath(out_dir)
    
    # Create temporary output names/directories
    n = 10000 # maximum N for random number generator
    tmp_out_dir = os.path.join(bids_out_dir, f"sub-{sub}", 'tmp_dir' + str(random.randint(0, n)))
    tmp_basename = 'tmp_basename' + str(random.randint(0, n))
    
    if not os.path.exists(tmp_out_dir):
        os.makedirs(tmp_out_dir)

    # Convert image file
    # Check file extension in file
    if '.nii.gz' in file:
        nii_file = utils.cp_file(file, tmp_out_dir, tmp_basename)
        [path,filename,ext] = utils.file_parts(file)
        json_file = os.path.join(path,filename + '.json')
        bval = os.path.join(path,filename + '.bval*')
        bvec = os.path.join(path,filename + '.bvec*')
        try:
            json_file = utils.cp_file(json_file, tmp_out_dir, tmp_basename)
            bval = utils.cp_file(bval, tmp_out_dir, tmp_basename)
            bvec = utils.cp_file(bvec, tmp_out_dir, tmp_basename)
        except FileNotFoundError:
            json_file = os.path.join(tmp_out_dir, tmp_basename + '.json')
            bval = os.path.join(tmp_out_dir, tmp_basename + '.bval')
            bvec = os.path.join(tmp_out_dir, tmp_basename + '.bvec')
            pass
    elif '.nii' in file:
        nii_file = utils.cp_file(file, tmp_out_dir, tmp_basename)
        nii_file = utils.gzip_file(nii_file)
        [path,filename,ext] = utils.file_parts(file)
        json_file = os.path.join(path,filename + '.json')
        bval = os.path.join(path,filename + '.bval*')
        bvec = os.path.join(path,filename + '.bvec*')
        try:
            json_file = utils.cp_file(json_file, tmp_out_dir, tmp_basename)
            bval = utils.cp_file(bval, tmp_out_dir, tmp_basename)
            bvec = utils.cp_file(bvec, tmp_out_dir, tmp_basename)
        except FileNotFoundError:
            json_file = os.path.join(tmp_out_dir, tmp_basename + '.json')
            bval = os.path.join(tmp_out_dir, tmp_basename + '.bval')
            bvec = os.path.join(tmp_out_dir, tmp_basename + '.bvec')
            pass
    elif '.dcm' in file or '.PAR' in file:
        [nii_file, json_file, bval, bvec] = utils.convert_dwi(file,tmp_out_dir,tmp_basename)
        # Decide if file is DWI or single-band reference
        num_frames = get_num_frames(nii_file)
        if num_frames == 1:
            scan = 'sbref'
            bval = ""
            bvec = ""
    else:
        [nii_file, json_file, bval, bvec] = utils.convert_dwi(file,tmp_out_dir,tmp_basename)
        # Decide if file is DWI or single-band reference
        num_frames = get_num_frames(nii_file)
        if num_frames == 1:
            scan = 'sbref'
            bval = ""
            bvec = ""
    
    # Get additional sequence/modality parameters
    if os.path.exists(json_file) and os.path.exists(bval):
        meta_dict_params = get_data_params(file, json_file, bval)
    elif os.path.exists(bval):
        tmp_json = ""
        meta_dict_params = get_data_params(file, tmp_json, bval)
    elif os.path.exists(json_file):
        tmp_bval = ""
        meta_dict_params = get_data_params(file, json_file, tmp_bval)
    else:
        tmp_json = ""
        tmp_bval = ""
        meta_dict_params = get_data_params(file, tmp_json, tmp_bval)
    
    # Update JSON file
    info = dict()
    info = utils.dict_multi_update(info,**meta_dict_params)
    info = utils.dict_multi_update(info,**meta_dict_com)
    info = utils.dict_multi_update(info,**meta_dict_dwi)
    
    json_file = utils.update_json(json_file,info)
    
    nii_file = os.path.abspath(nii_file)
    json_file = os.path.abspath(json_file)
    
    if bval and bvec:
        bval = os.path.abspath(bval)
        bvec = os.path.abspath(bvec)

    # Query dictionary for acquisition/naming keys
    try:
        acq = info['acq']
    except KeyError:
        acq = ""
        pass
    try:
        direction = info['dir']
    except KeyError:
        direction = ""
        pass
    
    # Non-standard acquisition/naming keys
    # Used in order to differentiate between DWI scans for multiple bvalues
    try:
        bvals = info['bval']
    except KeyError:
        bvals = list()
        if scan == 'sbref':
            bvals = 0
        pass
    try:
        echo_time = info['EchoTime']
    except KeyError:
        echo_time = ""
        pass
    
    # Create output filename    
    out_name = f"sub-{sub}" + f"_ses-{ses}"
    name_run_dict = dict()
    
    if bvals:
        vals = ""
        for val in bvals:
            vals = vals + 'b' + str(int(val))
    elif bvals == 0:
        vals = 'b0'
    
    if bvals and acq and echo_time:
        out_name = out_name + f"_acq-{acq}{vals}TE{echo_time}"
        tmp_dict = {"acq":f"{acq}{vals}TE{echo_time}"}
    elif bvals and acq:
        out_name = out_name + f"_acq-{acq}{vals}"
        tmp_dict = {"acq":f"{acq}{vals}"}
    elif bvals and echo_time:
        out_name = out_name + f"_acq-{vals}TE{echo_time}"
        tmp_dict = {"acq":f"{vals}TE{echo_time}"}
    elif acq and echo_time:
        out_name = out_name + f"_acq-{acq}TE{echo_time}"
        tmp_dict = {"acq":f"{acq}TE{echo_time}"}
    elif acq:
        out_name = out_name + f"_acq-{acq}"
        tmp_dict = {"acq":f"{acq}"}
    elif bvals:
        out_name = out_name + f"_acq-{vals}"
        tmp_dict = {"acq":f"{vals}"}
    elif echo_time:
        out_name = out_name + f"_acq-TE{echo_time}"
        tmp_dict = {"acq":f"TE{echo_time}"}
    else:
        tmp_dict = dict()
        
    name_run_dict.update(tmp_dict)

    if direction:
        out_name = out_name + f"_dir-{direction}"
        tmp_dict = {"dirs":f"{direction}"}
        name_run_dict.update(tmp_dict)
        
    # Get Run number
    run = utils.get_num_runs(out_dir, scan=scan, **name_run_dict)
    run = '{:02}'.format(run)

    if run:
        out_name = out_name + f"_run-{run}"

    out_name = out_name + f"_{scan}"

    out_nii = os.path.join(out_dir, out_name + '.nii.gz')
    out_json = os.path.join(out_dir, out_name + '.json')
    
    out_bval = os.path.join(out_dir, out_name + '.bval')
    out_bvec = os.path.join(out_dir, out_name + '.bvec')

    os.rename(nii_file, out_nii)
    os.rename(json_file, out_json)
    
    if bval:
        os.rename(bval,out_bval)
        
    if bvec:
        os.rename(bvec,out_bvec)

    # remove temporary directory and leftover files
    shutil.rmtree(tmp_out_dir)
    
    if bval and bvec:
        return out_nii,out_json,out_bval,out_bvec
    elif not bval and bvec:
        return out_nii,out_json

In [24]:
info = dict()
scan = 'sbref'

In [25]:
try:
    bvals = info['bval']
except KeyError:
    bvals = list()
    if scan == 'sbref':
        bvals = 0

In [26]:
scan

'sbref'

In [27]:
if bvals:
    vals = ""
    for val in bvals:
        vals = vals + 'b' + str(int(val))
elif bvals == 0:
    vals = 'b0'

In [28]:
vals

'b0'

In [29]:
if vals:
    print(vals)

b0


In [30]:
if bvals == 0:
    print(True)

True


In [31]:
bvals = list()

In [32]:
if bvals:
    print(True)

In [24]:
bvals = 0; scan = 'sbref'; i = 2

In [27]:
echo = 0.088

In [28]:
echo = int(echo * 1000)
echo

88

In [29]:
j = "/Users/brac4g/Desktop/convsauce/BIDS.new/sub-P08/ses-001/dwi/sub-P08_ses-001_acq-b0_dir-PA_run-01_dwi.json"
j2 = "/Users/brac4g/Desktop/convsauce/BIDS.new/sub-P08/ses-001/dwi/sub-P08_ses-001_acq-b0_dir-PA_run-01_dwi2.json"

In [31]:
def read_json(json_file):
    '''
    Reads JavaScript Object Notation (JSON) file.
    
    Arguments:
        json_file (string): Input file
        
    Returns: 
        data (dict): Dictionary of key mapped items in JSON file
    '''
    
    # Read JSON file
    # Try-Except statement has empty exception as JSONDecodeError is not a valid exception to pass, 
    # thus throwing a name error
    try:
        with open(json_file) as file:
            data = json.load(file)
    except:
        data = dict()
        
    return data

In [28]:
read_json(j)

{'Manufacturer': 'Philips',
 'PatientPosition': 'HFS',
 'SeriesDescription': 'ImageMRSERIES',
 'ProtocolName': 'WIP6DIRB0A',
 'SeriesNumber': 5,
 'AcquisitionNumber': 5,
 'ImageComments': 'IRC287H',
 'ConversionComments': 'y',
 'PhilipsRescaleSlope': 2.01465,
 'PhilipsRescaleIntercept': 0,
 'PhilipsScaleSlope': 0.028553,
 'UsePhilipsFloatNotDisplayScaling': 1,
 'EchoTime': 0.088,
 'RepetitionTime': 15.94,
 'ReconMatrixPE': 80,
 'ImageOrientationPatientDICOM': [0.982342,
  0.0955086,
  0.160883,
  -0.0999558,
  0.994796,
  0.0197614],
 'ConversionSoftware': 'dcm2niix',
 'ConversionSoftwareVersion': 'v1.0.20190720  (JP2:OpenJPEG) (JP-LS:CharLS) Clang8.1.0',
 'WaterFatShift': 22.516,
 'ParallelAcquisitionTechnique': 'SENSE',
 'ParallelReductionFactorInPlane': 1.0,
 'MultibandAccelerationFactor': 1,
 'EffectiveEchoSpacing': 0.6481812005573276,
 'TotalReadoutTime': 0.051206314844028884,
 'AcquisitionDuration': 97.8,
 'EchoTrainLength': 79,
 'SourceDataFormat': 'PAR REC',
 'ManufacturersMode

In [32]:
read_json(j2)

{}

In [24]:
def data_to_bids_anat(bids_out_dir, file, sub, scan, meta_dict_com=dict(), meta_dict_anat=dict(), ses=1, scan_type='anat'):
    '''
    Renames converted NifTi-2 files to conform with the BIDS naming convension (in the case of anatomical files).
    This function accepts any image file (DICOM, PAR REC, and NifTi-2). If the image file is a raw data file (e.g. DICOM, PAR REC)
    it is converted to NifTi first, then renamed. The output BIDS directory need not exist at runtime.
    
    Arguments:
        bids_out_dir (string): Path to output BIDS directory. 
        file (string): Filepath to image file.
        sub (int or string): Subject ID
        scan (string): Modality (e.g. T1w, T2w, or SWI)
        meta_dict_com (dict): Metadata dictionary for common image metadata
        meta_dict_anat (dict): Metadata dictionary for common anatomical image specific metadata
        ses (int or string): Session ID
        scan_type (string): BIDS sub-directory scan type. Valid options include, but are not limited to: anat (default), func, fmap, dwi, etc.
        
    Returns:
        out_nii (string): Absolute filepath to gzipped output NifTi-2 file
        out_json (string): Absolute filepath to corresponding JSON file
    '''

    # Use try-except statement here in the case of invalid/incomplete image files that will throw errors in dcm2niix
    try:
        # Create Output Directory Variables
        # Zeropad subject ID if possible
        try:
            ses = '{:03}'.format(int(ses))
        except ValueError:
            pass
        # Zeropad session ID if possible
        try:
            ses = '{:03}'.format(int(ses))
        except ValueError:
            pass

        out_dir = os.path.join(bids_out_dir, f"sub-{sub}", f"ses-{ses}", f"{scan_type}")

        # Make output directory
        if not os.path.exists(out_dir):
            os.makedirs(out_dir)

        # Get absolute filepaths
        bids_out_dir = os.path.abspath(bids_out_dir)
        out_dir = os.path.abspath(out_dir)

        # Create temporary output names/directories
        n = 10000 # maximum N for random number generator
        tmp_out_dir = os.path.join(bids_out_dir, f"sub-{sub}", 'tmp_dir' + str(random.randint(0, n)))
        tmp_basename = 'tmp_basename' + str(random.randint(0, n))

        if not os.path.exists(tmp_out_dir):
            os.makedirs(tmp_out_dir)

        # Convert image file
        # Check file extension in file
        if '.nii.gz' in file:
            nii_file = utils.cp_file(file, tmp_out_dir, tmp_basename)
            [path,filename,ext] = utils.file_parts(file)
            json_file = os.path.join(path,filename + '.json')
            try:
                json_file = utils.cp_file(json_file, tmp_out_dir, tmp_basename)
            except FileNotFoundError:
                json_file = os.path.join(tmp_out_dir, tmp_basename + '.json')
                pass
        elif '.nii' in file:
            nii_file = utils.cp_file(file, tmp_out_dir, tmp_basename)
            nii_file = utils.gzip_file(nii_file)
            [path,filename,ext] = utils.file_parts(file)
            json_file = os.path.join(path,filename + '.json')
            try:
                json_file = utils.cp_file(json_file, tmp_out_dir, tmp_basename)
            except FileNotFoundError:
                json_file = os.path.join(tmp_out_dir, tmp_basename + '.json')
                pass
        elif '.dcm' in file or '.PAR' in file:
            [nii_file, json_file] = utils.convert_anat(file,tmp_out_dir,tmp_basename)
        else:
            [nii_file, json_file] = utils.convert_anat(file,tmp_out_dir,tmp_basename)

        # Get additional sequence/modality parameters
        if os.path.exists(json_file):
            meta_dict_params = get_data_params(file, json_file)
        else:
            tmp_json = ""
            meta_dict_params = get_data_params(file, tmp_json)

        # Update JSON file
        info = dict()
        info = utils.dict_multi_update(info,**meta_dict_params)
        info = utils.dict_multi_update(info,**meta_dict_com)
        info = utils.dict_multi_update(info,**meta_dict_anat)

        json_file = utils.update_json(json_file,info)

        nii_file = os.path.abspath(nii_file)
        json_file = os.path.abspath(json_file)

        info = dict()
        info = utils.read_json(json_file)

        # Append w to T1/T2 if not already done
        if scan in 'T1' or scan in 'T2':
            scan = scan + 'w'

        # Query dictionary for acquisition/naming keys
        try:
            acq = info['acq']
        except KeyError:
            acq = ""
            pass
        try:
            ce = info['ce']
        except KeyError:
            ce = ""
            pass
        try:
            rec = info['rec']
        except KeyError:
            rec = ""
            pass

        # Create output filename
        out_name = f"sub-{sub}" + f"_ses-{sub}"
        name_run_dict = dict()

        if acq:
            out_name = out_name + f"_acq-{acq}"
            tmp_dict = {"acq":f"{acq}"}
            name_run_dict.update(tmp_dict)

        if ce:
            out_name = out_name + f"_ce-{ce}"
            tmp_dict = {"ce":f"{ce}"}
            name_run_dict.update(tmp_dict)

        if rec:
            out_name = out_name + f"_rec-{rec}"
            tmp_dict = {"rec":f"{rec}"}
            name_run_dict.update(tmp_dict)

        # Get Run number
        run = utils.get_num_runs(out_dir, scan=scan, **name_run_dict)
        run = '{:02}'.format(run)

        if run:
            out_name = out_name + f"_run-{run}"

        out_name = out_name + f"_{scan}"


        out_nii = os.path.join(out_dir, out_name + '.nii.gz')
        out_json = os.path.join(out_dir, out_name + '.json')

        os.rename(nii_file, out_nii)
        os.rename(json_file, out_json)

        # remove temporary directory and leftover files
        shutil.rmtree(tmp_out_dir)

        return out_nii,out_json
    except FileNotFoundError:
        print(f"Error: unable to convert {file}")
        pass

In [26]:
def data_to_bids_func(bids_out_dir, file, sub, scan, task = 'rest', meta_dict_com=dict(), meta_dict_func=dict(), ses=1, scan_type='func'):
    '''
    Renames converted NifTi-2 files to conform with the BIDS naming convension (in the case of functional files).
    This function accepts any image file (DICOM, PAR REC, and NifTi-2). If the image file is a raw data file (e.g. DICOM, PAR REC)
    it is converted to NifTi first, then renamed. The output BIDS directory need not exist at runtime.
    
    Arguments:
        bids_out_dir (string): Path to output BIDS directory. 
        file (string): Filepath to image file.
        sub (int or string): Subject ID
        scan (string): Modality (e.g. bold or cbv)
        task (string): Task for the fMR image data
        meta_dict_com (dict): Metadata dictionary for common image metadata
        meta_dict_func (dict): Metadata dictionary for common functional image specific metadata
        ses (int or string): Session ID
        scan_type (string): BIDS sub-directory scan type. Valid options include, but are not limited to: anat, func (default), fmap, dwi, etc.
        
    Returns:
        out_nii (string): Absolute filepath to gzipped output 4D NifTi-2 file
        out_json (string): Absolute filepath to corresponding JSON file
    '''

    # Use try-except statement here in the case of invalid/incomplete image files that will throw errors in dcm2niix
    try:
        # Create Output Directory Variables
        # Zeropad subject ID if possible
        try:
            ses = '{:03}'.format(int(ses))
        except ValueError:
            pass
        # Zeropad session ID if possible
        try:
            ses = '{:03}'.format(int(ses))
        except ValueError:
            pass

        out_dir = os.path.join(bids_out_dir, f"sub-{sub}", f"ses-{ses}", f"{scan_type}")

        # Make output directory
        if not os.path.exists(out_dir):
            os.makedirs(out_dir)

        # Get absolute filepaths
        bids_out_dir = os.path.abspath(bids_out_dir)
        out_dir = os.path.abspath(out_dir)

        # Create temporary output names/directories
        n = 10000 # maximum N for random number generator
        tmp_out_dir = os.path.join(bids_out_dir, f"sub-{sub}", 'tmp_dir' + str(random.randint(0, n)))
        tmp_basename = 'tmp_basename' + str(random.randint(0, n))

        if not os.path.exists(tmp_out_dir):
            os.makedirs(tmp_out_dir)

        # Convert image file
        # Check file extension in file
        if '.nii.gz' in file:
            nii_file = utils.cp_file(file, tmp_out_dir, tmp_basename)
            [path,filename,ext] = utils.file_parts(file)
            json_file = os.path.join(path,filename + '.json')
            try:
                json_file = utils.cp_file(json_file, tmp_out_dir, tmp_basename)
            except FileNotFoundError:
                json_file = os.path.join(tmp_out_dir, tmp_basename + '.json')
                pass
        elif '.nii' in file:
            nii_file = utils.cp_file(file, tmp_out_dir, tmp_basename)
            nii_file = utils.gzip_file(nii_file)
            [path,filename,ext] = utils.file_parts(file)
            json_file = os.path.join(path,filename + '.json')
            try:
                json_file = utils.cp_file(json_file, tmp_out_dir, tmp_basename)
            except FileNotFoundError:
                json_file = os.path.join(tmp_out_dir, tmp_basename + '.json')
                pass
        elif '.dcm' in file or '.PAR' in file:
            [nii_file, json_file] = utils.convert_anat(file,tmp_out_dir,tmp_basename)
        else:
            [nii_file, json_file] = utils.convert_anat(file,tmp_out_dir,tmp_basename)

        # Get additional sequence/modality parameters
        if os.path.exists(json_file):
            meta_dict_params = get_data_params(file, json_file)
        else:
            tmp_json = ""
            meta_dict_params = get_data_params(file, tmp_json)

        # Update JSON file
        info = dict()
        info = utils.dict_multi_update(info,**meta_dict_params)
        info = utils.dict_multi_update(info,**meta_dict_com)
        info = utils.dict_multi_update(info,**meta_dict_func)

        json_file = utils.update_json(json_file,info)

        nii_file = os.path.abspath(nii_file)
        json_file = os.path.abspath(json_file)

        info = dict()
        info = utils.read_json(json_file)

        # Decide if file is 4D timeseries or single-band reference
        num_frames = get_num_frames(nii_file)
        if num_frames == 1:
            scan = 'sbref'

        # Query dictionary for acquisition/naming keys
        try:
            acq = info['acq']
        except KeyError:
            acq = ""
            pass
        try:
            ce = info['ce']
        except KeyError:
            ce = ""
            pass
        try:
            direction = info['dir']
        except KeyError:
            direction = ""
            pass
        try:
            rec = info['rec']
        except KeyError:
            rec = ""
            pass
        try:
            echo = info['echo']
        except KeyError:
            echo = ""
            pass

        # Create output filename    
        out_name = f"sub-{sub}" + f"_ses-{ses}" + f"_task-{task}"

        name_run_dict = dict()
        tmp_dict = {"task":f"{task}"}
        name_run_dict.update(tmp_dict)

        if acq:
            out_name = out_name + f"_acq-{acq}"
            tmp_dict = {"acq":f"{acq}"}
            name_run_dict.update(tmp_dict)

        if ce:
            out_name = out_name + f"_ce-{ce}"
            tmp_dict = {"ce":f"{ce}"}
            name_run_dict.update(tmp_dict)

        if direction:
            out_name = out_name + f"_dir-{direction}"
            tmp_dict = {"dirs":f"{direction}"}
            name_run_dict.update(tmp_dict)

        if rec:
            out_name = out_name + f"_rec-{rec}"
            tmp_dict = {"rec":f"{rec}"}
            name_run_dict.update(tmp_dict)

        if echo:
            tmp_dict = {"echo":f"{echo}"}
            name_run_dict.update(tmp_dict)

        # Get Run number
        run = utils.get_num_runs(out_dir, scan=scan, **name_run_dict)
        run = '{:02}'.format(run)

        if run:
            out_name = out_name + f"_run-{run}"

        if echo:
            out_name = out_name + f"_echo-{echo}"

        out_name = out_name + f"_{scan}"


        out_nii = os.path.join(out_dir, out_name + '.nii.gz')
        out_json = os.path.join(out_dir, out_name + '.json')

        os.rename(nii_file, out_nii)
        os.rename(json_file, out_json)

        # remove temporary directory and leftover files
        shutil.rmtree(tmp_out_dir)

        return out_nii,out_json
    except FileNotFoundError:
        print(f"Error: unable to convert {file}")
        pass

In [27]:
def data_to_bids_fmap(bids_out_dir, file, sub, scan='fieldmap', meta_dict_com=dict(), meta_dict_fmap=dict(), ses=1, scan_type='fmap'):
    '''
    Renames converted NifTi-2 files to conform with the BIDS naming convension (in the case of fieldmap files).
    This function accepts any image file (DICOM, PAR REC, and NifTi-2). If the image file is a raw data file (e.g. DICOM, PAR REC)
    it is converted to NifTi first, then renamed. The output BIDS directory need not exist at runtime.
    
    N.B.: This function is mainly designed to handle fieldmap data case 3 from bids-specifications document. Furhter support for 
    the additional cases requires test/validation data. 
    BIDS-specifications document located here: 
    https://github.com/bids-standard/bids-specification/blob/master/src/04-modality-specific-files/01-magnetic-resonance-imaging-data.md
    
    Arguments:
        bids_out_dir (string): Path to output BIDS directory. 
        file (string): Filepath to image file.
        sub (int or string): Subject ID
        scan (string): Modality (e.g. fieldmap, magnitude, or phasediff)
        meta_dict_com (dict): Metadata dictionary for common image metadata
        meta_dict_fmap (dict): Metadata dictionary for common fieldmap image specific metadata
        ses (int or string): Session ID
        scan_type (string): BIDS sub-directory scan type. Valid options include, but are not limited to: anat, func, fmap (default), dwi, etc.
        
    Returns:
        out_nii_fmap (string): Absolute filepath to gzipped output NifTi-2 fieldmap image file
        out_nii_mag (string): Absolute filepath to gzipped output NifTi-2 magnitude image file
        out_json_fmap (string): Absolute filepath to correspond fieldmap image JSON sidecare
        out_json_mag (string): Absolute filepath to correspond magnitude image JSON sidecare
    '''

    # Use try-except statement here in the case of invalid/incomplete image files that will throw errors in dcm2niix
    try:
        # Create Output Directory Variables
        # Zeropad subject ID if possible
        try:
            ses = '{:03}'.format(int(ses))
        except ValueError:
            pass
        # Zeropad session ID if possible
        try:
            ses = '{:03}'.format(int(ses))
        except ValueError:
            pass

        out_dir = os.path.join(bids_out_dir, f"sub-{sub}", f"ses-{ses}", f"{scan_type}")

        # Make output directory
        if not os.path.exists(out_dir):
            os.makedirs(out_dir)

        # Get absolute filepaths
        bids_out_dir = os.path.abspath(bids_out_dir)
        out_dir = os.path.abspath(out_dir)

        # Create temporary output names/directories
        n = 10000 # maximum N for random number generator
        tmp_out_dir = os.path.join(bids_out_dir, f"sub-{sub}", 'tmp_dir' + str(random.randint(0, n)))
        tmp_basename = 'tmp_basename' + str(random.randint(0, n))

        if not os.path.exists(tmp_out_dir):
            os.makedirs(tmp_out_dir)

        # Convert image file
        # Check file extension in file
        if '.nii.gz' in file:
            nii_file = utils.cp_file(file, tmp_out_dir, tmp_basename)
            [path,filename,ext] = utils.file_parts(file)
            json_file = os.path.join(path,filename + '.json')
            try:
                json_file = utils.cp_file(json_file, tmp_out_dir, tmp_basename)
            except FileNotFoundError:
                json_file = os.path.join(tmp_out_dir, tmp_basename + '.json')
                pass
        elif '.nii' in file:
            nii_file = utils.cp_file(file, tmp_out_dir, tmp_basename)
            nii_file = utils.gzip_file(nii_file)
            [path,filename,ext] = utils.file_parts(file)
            json_file = os.path.join(path,filename + '.json')
            try:
                json_file = utils.cp_file(json_file, tmp_out_dir, tmp_basename)
            except FileNotFoundError:
                json_file = os.path.join(tmp_out_dir, tmp_basename + '.json')
                pass
        elif '.dcm' in file or '.PAR' in file:
            [nii_fmap, json_fmap, nii_mag, json_mag] = utils.convert_fmap(file,tmp_out_dir,tmp_basename)
        else:
            [nii_fmap, json_fmap, nii_mag, json_mag] = utils.convert_fmap(file,tmp_out_dir,tmp_basename)

        # Get additional sequence/modality parameters
        if os.path.exists(json_fmap):
            meta_dict_params = get_data_params(file, json_fmap)
        else:
            tmp_json = ""
            meta_dict_params = get_data_params(file, tmp_json)

        # Update JSON file
        info = dict()
        info = utils.dict_multi_update(info,**meta_dict_params)
        info = utils.dict_multi_update(info,**meta_dict_com)
        info = utils.dict_multi_update(info,**meta_dict_fmap)

        json_fmap = utils.update_json(json_fmap,info)
        json_mag = utils.update_json(json_mag,info)

        nii_fmap = os.path.abspath(nii_fmap)
        nii_mag = os.path.abspath(nii_mag)

        json_fmap = os.path.abspath(json_fmap)
        json_mag = os.path.abspath(json_mag)

        info = dict()
        info = utils.read_json(json_fmap)

        # Query dictionary for acquisition/naming keys
        try:
            acq = info['acq']
        except KeyError:
            acq = ""
            pass

        # Create output filename    
        out_name = f"sub-{sub}" + f"_ses-{ses}"
        name_run_dict = dict()

        if acq:
            out_name = out_name + f"_acq-{acq}"
            tmp_dict = {"acq":f"{acq}"}
            name_run_dict.update(tmp_dict)

        # Get Run number
        run = utils.get_num_runs(out_dir, scan=scan, **name_run_dict)
        run = '{:02}'.format(run)

        if run:
            out_name = out_name + f"_run-{run}"

        out_name = out_name + f"_{scan}"

        out_nii_fmap = os.path.join(out_dir, out_name + '_fieldmap' + '.nii.gz')
        out_nii_mag = os.path.join(out_dir, out_name + '_magnitude' + '.nii.gz')

        out_json_fmap = os.path.join(out_dir, out_name + '_fieldmap' + '.json')
        out_json_mag = os.path.join(out_dir, out_name + '_magnitude' + '.json')

        os.rename(nii_fmap, out_nii_fmap)
        os.rename(nii_mag, out_nii_mag)

        os.rename(json_fmap, out_json_fmap)
        os.rename(json_mag, out_json_mag)

        # Remove temporary directory and leftover files
        shutil.rmtree(tmp_out_dir)

        return out_nii_fmap, out_nii_mag, out_json_fmap, out_json_mag
    except FileNotFoundError:
        print(f"Error: unable to convert {file}")
        pass

In [28]:
def data_to_bids_dwi(bids_out_dir, file, sub, scan='dwi', meta_dict_com=dict(), meta_dict_dwi=dict(), ses=1, scan_type='dwi'):
    '''
    Renames converted NifTi-2 files to conform with the BIDS naming convension (in the case of diffuion image files).
    This function accepts any image file (DICOM, PAR REC, and NifTi-2). If the image file is a raw data file (e.g. DICOM, PAR REC)
    it is converted to NifTi first, then renamed. The output BIDS directory need not exist at runtime. If the original
    data format is NifTi, bval and bvec files will be copied over should they exist, otherwise, they will not be
    generated.
    
    Arguments:
        bids_out_dir (string): Path to output BIDS directory. 
        file (string): Filepath to image file.
        sub (int or string): Subject ID
        scan (string): Modality (e.g. dwi, dki, etc)
        meta_dict_com (dict): Metadata dictionary for common image metadata
        meta_dict_dwi (dict): Metadata dictionary for common diffusion image specific metadata
        ses (int or string): Session ID
        scan_type (string): BIDS sub-directory scan type. Valid options include, but are not limited to: anat, func, fmap, dwi (default), etc.
        
    Returns:
        out_nii (string): Absolute filepath to gzipped output diffusion weighted NifTi-2 file
        out_json (string): Absolute filepath to corresponding JSON file
        out_bval (string): Absolute filepath to corresponding b-values file
        out_bvec (string): Absolute filepath to corresponding b-vectors file
    '''

    # Use try-except statement here in the case of invalid/incomplete image files that will throw errors in dcm2niix
    try:
        # Create Output Directory Variables
        # Zeropad subject ID if possible
        try:
            ses = '{:03}'.format(int(ses))
        except ValueError:
            pass
        # Zeropad session ID if possible
        try:
            ses = '{:03}'.format(int(ses))
        except ValueError:
            pass

        out_dir = os.path.join(bids_out_dir, f"sub-{sub}", f"ses-{ses}", f"{scan_type}")

        # Make output directory
        if not os.path.exists(out_dir):
            os.makedirs(out_dir)

        # Get absolute filepaths
        bids_out_dir = os.path.abspath(bids_out_dir)
        out_dir = os.path.abspath(out_dir)

        # Create temporary output names/directories
        n = 10000 # maximum N for random number generator
        tmp_out_dir = os.path.join(bids_out_dir, f"sub-{sub}", 'tmp_dir' + str(random.randint(0, n)))
        tmp_basename = 'tmp_basename' + str(random.randint(0, n))

        if not os.path.exists(tmp_out_dir):
            os.makedirs(tmp_out_dir)

        # Convert image file
        # Check file extension in file
        if '.nii.gz' in file:
            nii_file = utils.cp_file(file, tmp_out_dir, tmp_basename)
            [path,filename,ext] = utils.file_parts(file)
            json_file = os.path.join(path,filename + '.json')
            bval = os.path.join(path,filename + '.bval*')
            bvec = os.path.join(path,filename + '.bvec*')
            try:
                json_file = utils.cp_file(json_file, tmp_out_dir, tmp_basename)
                # Decide if file is DWI or single-band reference
                num_frames = get_num_frames(nii_file)
                if num_frames == 1:
                    scan = 'sbref'; bval = ""; bvec = ""
                else:
                    bval = utils.cp_file(bval, tmp_out_dir, tmp_basename)
                    bvec = utils.cp_file(bvec, tmp_out_dir, tmp_basename)
            except FileNotFoundError:
                json_file = os.path.join(tmp_out_dir, tmp_basename + '.json')
                bval = os.path.join(tmp_out_dir, tmp_basename + '.bval')
                bvec = os.path.join(tmp_out_dir, tmp_basename + '.bvec')
                # Decide if file is DWI or single-band reference
                num_frames = get_num_frames(nii_file)
                if num_frames == 1:
                    scan = 'sbref'; bval = ""; bvec = ""
                pass
        elif '.nii' in file:
            nii_file = utils.cp_file(file, tmp_out_dir, tmp_basename)
            nii_file = utils.gzip_file(nii_file)
            [path,filename,ext] = utils.file_parts(file)
            json_file = os.path.join(path,filename + '.json')
            bval = os.path.join(path,filename + '.bval*')
            bvec = os.path.join(path,filename + '.bvec*')
            try:
                json_file = utils.cp_file(json_file, tmp_out_dir, tmp_basename)
                # Decide if file is DWI or single-band reference
                num_frames = get_num_frames(nii_file)
                if num_frames == 1:
                    scan = 'sbref'; bval = ""; bvec = ""
                else:
                    bval = utils.cp_file(bval, tmp_out_dir, tmp_basename)
                    bvec = utils.cp_file(bvec, tmp_out_dir, tmp_basename)
            except FileNotFoundError:
                json_file = os.path.join(tmp_out_dir, tmp_basename + '.json')
                bval = os.path.join(tmp_out_dir, tmp_basename + '.bval')
                bvec = os.path.join(tmp_out_dir, tmp_basename + '.bvec')
                # Decide if file is DWI or single-band reference
                num_frames = get_num_frames(nii_file)
                if num_frames == 1:
                    scan = 'sbref'; bval = ""; bvec = ""
                pass
        elif '.dcm' in file or '.PAR' in file:
            [nii_file, json_file, bval, bvec] = utils.convert_dwi(file,tmp_out_dir,tmp_basename)
            # Decide if file is DWI or single-band reference
            num_frames = get_num_frames(nii_file)
            if num_frames == 1:
                scan = 'sbref'; bval = ""; bvec = ""
        else:
            [nii_file, json_file, bval, bvec] = utils.convert_dwi(file,tmp_out_dir,tmp_basename)
            # Decide if file is DWI or single-band reference
            num_frames = get_num_frames(nii_file)
            if num_frames == 1:
                scan = 'sbref'; bval = ""; bvec = ""

        # Get additional sequence/modality parameters
        if os.path.exists(json_file) and os.path.exists(bval):
            meta_dict_params = get_data_params(file, json_file, bval)
        elif os.path.exists(bval):
            tmp_json = ""
            meta_dict_params = get_data_params(file, tmp_json, bval)
        elif os.path.exists(json_file):
            tmp_bval = ""
            meta_dict_params = get_data_params(file, json_file, tmp_bval)
        else:
            tmp_json = ""
            tmp_bval = ""
            meta_dict_params = get_data_params(file, tmp_json, tmp_bval)

        # Update JSON file
        info = dict()
        info = utils.dict_multi_update(info,**meta_dict_params)
        info = utils.dict_multi_update(info,**meta_dict_com)
        info = utils.dict_multi_update(info,**meta_dict_dwi)

        json_file = utils.update_json(json_file,info)

        nii_file = os.path.abspath(nii_file)
        json_file = os.path.abspath(json_file)

        info = dict()
        info = utils.read_json(json_file)

        if bval and bvec:
            bval = os.path.abspath(bval)
            bvec = os.path.abspath(bvec)

        # Query dictionary for acquisition/naming keys
        try:
            acq = info['acq']
        except KeyError:
            acq = ""
            pass
        try:
            direction = info['dir']
        except KeyError:
            direction = ""
            pass

        # Non-standard acquisition/naming keys
        # Used in order to differentiate between DWI scans for multiple bvalues
        try:
            bvals = info['bval']
        except KeyError:
            bvals = list()
            pass
        try:
            echo_time = info['EchoTime']
            echo_time = int(echo_time * 1000)
        except KeyError:
            echo_time = ""
            pass

        # Create output filename    
        out_name = f"sub-{sub}" + f"_ses-{ses}"
        name_run_dict = dict()

        if bvals:
            vals = ""
            for val in bvals:
                vals = vals + 'b' + str(int(val))
        else:
            vals = 'b0'

        if vals and acq and echo_time:
            out_name = out_name + f"_acq-{acq}{vals}TE{echo_time}"
            tmp_dict = {"acq":f"{acq}{vals}TE{echo_time}"}
        elif vals and acq:
            out_name = out_name + f"_acq-{acq}{vals}"
            tmp_dict = {"acq":f"{acq}{vals}"}
        elif vals and echo_time:
            out_name = out_name + f"_acq-{vals}TE{echo_time}"
            tmp_dict = {"acq":f"{vals}TE{echo_time}"}
        elif acq and echo_time:
            out_name = out_name + f"_acq-{acq}TE{echo_time}"
            tmp_dict = {"acq":f"{acq}TE{echo_time}"}
        elif acq:
            out_name = out_name + f"_acq-{acq}"
            tmp_dict = {"acq":f"{acq}"}
        elif vals:
            out_name = out_name + f"_acq-{vals}"
            tmp_dict = {"acq":f"{vals}"}
        elif echo_time:
            out_name = out_name + f"_acq-TE{echo_time}"
            tmp_dict = {"acq":f"TE{echo_time}"}
        else:
            tmp_dict = dict()

        name_run_dict.update(tmp_dict)

        if direction:
            out_name = out_name + f"_dir-{direction}"
            tmp_dict = {"dirs":f"{direction}"}
            name_run_dict.update(tmp_dict)

        # Get Run number
        run = utils.get_num_runs(out_dir, scan=scan, **name_run_dict)
        run = '{:02}'.format(run)

        if run:
            out_name = out_name + f"_run-{run}"

        out_name = out_name + f"_{scan}"

        out_nii = os.path.join(out_dir, out_name + '.nii.gz')
        out_json = os.path.join(out_dir, out_name + '.json')

        out_bval = os.path.join(out_dir, out_name + '.bval')
        out_bvec = os.path.join(out_dir, out_name + '.bvec')

        os.rename(nii_file, out_nii)
        os.rename(json_file, out_json)

        if bval:
            os.rename(bval,out_bval)

        if bvec:
            os.rename(bvec,out_bvec)

        # remove temporary directory and leftover files
        shutil.rmtree(tmp_out_dir)

        if bval and bvec:
            return out_nii,out_json,out_bval,out_bvec
        elif not bval and bvec:
            return out_nii,out_json
    except FileNotFoundError:
        print(f"Error: unable to convert {file}")
        pass

# Additional Feature Implementation

In [ ]:
def batch_convert(bids_out_dir,sub,file_list, search_dict, meta_dict=dict(), ses=1, keep_unknown=True,verbose=False):
    '''
    Batch conversion function for image files.
    
    Note: This function is still undergoing active development.

    Arguments:
        bids_out_dir (string): Output BIDS directory
        sub (int or string): Subject ID
        file (string): Source image filename with absolute filepath
        search_dict (dict): Nested dictionary from the 'read_config' function
        meta_dict (dict): Nested metadata dictionary
        ses (int or string): Session ID
        keep_unknown (bool): Convert modalities/scans which cannot be identified (default: True)
        verbose (bool): Prints the scan_type, modality, and search terms used (e.g. func - bold - rest - ['rest', 'FFE'])

    Returns: 
        None
    '''

    converted_files = list()
    
    for file in file_list:
        try:
            converted_files = convert_modality(bids_out_dir=bids_out_dir, sub=sub, file=file, search_dict=search_dict, meta_dict=meta_dict, ses=1, keep_unknown=True, verbose=False)
        except (SystemExit,FileNotFoundError):
            pass
    
    return converted_files

In [20]:
# look for PAR files

In [12]:
dir_ = os.path.join('/Users/brac4g/Desktop/convsauce/pat_dir','*')

In [13]:
file_ext = '.dcm'

In [14]:
dir_list = glob.glob(dir_,recursive=True)
dir_list

['/Users/brac4g/Desktop/convsauce/pat_dir/IRC287H-8',
 '/Users/brac4g/Desktop/convsauce/pat_dir/287H_C10']

In [25]:
for i in dir_list:
    for root,dirs,files in os.walk(i):
        print(root)
        print(dirs)
        # print(files)
#         if files:
#             print(root,dirs,files[0])
#             break
        

/Users/brac4g/Desktop/convsauce/pat_dir/IRC287H-8
['20171003']
/Users/brac4g/Desktop/convsauce/pat_dir/IRC287H-8/20171003
['0_DEFAULT_PS_SERIES_2017100310374358016', '303_CORONAL_2017100310262626000', '0_DEFAULT_PS_SERIES_2017100310463791022', '1701_WM_SV_PRESS_35_017100311174543840', '201_SAG_T1W_3D_Y_INNER_TI_1100_017100310184810020', '1101_rsfMRI_MB6_SENSE_1_fat_shift_P_017100310465322437']
/Users/brac4g/Desktop/convsauce/pat_dir/IRC287H-8/20171003/0_DEFAULT_PS_SERIES_2017100310374358016
[]
/Users/brac4g/Desktop/convsauce/pat_dir/IRC287H-8/20171003/303_CORONAL_2017100310262626000
[]
/Users/brac4g/Desktop/convsauce/pat_dir/IRC287H-8/20171003/0_DEFAULT_PS_SERIES_2017100310463791022
[]
/Users/brac4g/Desktop/convsauce/pat_dir/IRC287H-8/20171003/1701_WM_SV_PRESS_35_017100311174543840
[]
/Users/brac4g/Desktop/convsauce/pat_dir/IRC287H-8/20171003/201_SAG_T1W_3D_Y_INNER_TI_1100_017100310184810020
[]
/Users/brac4g/Desktop/convsauce/pat_dir/IRC287H-8/20171003/1101_rsfMRI_MB6_SENSE_1_fat_shift

In [31]:
def walk2(top, topdown=True, onerror=None, followlinks=False, maxdepth=None):
    
    import os
    import os.path as path
    
    islink, join, isdir = path.islink, path.join, path.isdir


    try:
        names = os.listdir(top)
    except(OSError, err):
        if onerror is not None:
            onerror(err)
        return

    dirs, nondirs = [], []
    for name in names:
        if isdir(join(top, name)):
            dirs.append(name)
        else:
            nondirs.append(name)

    if topdown:
        yield top, dirs, nondirs

    if maxdepth is None or maxdepth > 1:
        for name in dirs:
            new_path = join(top, name)
            if followlinks or not islink(new_path):
                for x in walk2(new_path, topdown, onerror, followlinks, None if maxdepth is None else maxdepth-1):
                    yield x
    if not topdown:
        yield top, dirs, nondirs

In [32]:
for i in dir_list:
    for root,dirs,files in walk2(i,maxdepth=5):
        # print(root)
        # print(dirs)
        # print(files)
        if files:
            print(root,dirs,files[0])
            break
        

/Users/brac4g/Desktop/convsauce/pat_dir/IRC287H-8/20171003/0_DEFAULT_PS_SERIES_2017100310374358016 [] PR0000000001.dcm
/Users/brac4g/Desktop/convsauce/pat_dir/287H_C10/NIFTI [] 287H_C10_SAG_4_5.nii.gz
